In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
import os
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True)
        )
        #(128, 4, 4)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )


    def forward(self, x):
        encoded = self.encoder(x)
        latent_vector = encoded 
        decoded = self.decoder(latent_vector)
        return decoded, latent_vector

In [ ]:
data_root = './places365_data'

dataset = datasets.Places365(
    root=data_root,
    split='val',              # Split: 'train-standard', 'train-challenge', or 'val'
    small=True,
    download=True,
    transform=transforms.ToTensor()
)

BATCH_SIZE = 1
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=16)


In [ ]:
EPOCHS = 5
LEARNING_RATE = 1e-3

model = AutoEncoder().to(device)
criterion = nn.MSELoss() 
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print(f"\nModel initialized and sent to {device}.")
print(f"Total parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print("-" * 30)

# --- 4B. The Training Function ---
def train_model(model, dataloader, criterion, optimizer, num_epochs):
    model.train()
    training_losses = []
    
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for batch_idx, (data, _) in enumerate(dataloader):
            data = data.to(device)
            reconstructed, _ = model(data)
            loss = criterion(reconstructed, data)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item() * data.size(0)

            if batch_idx % 10 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx}/{len(dataloader)}], Loss: {loss.item():.6f}')

        avg_epoch_loss = epoch_loss / len(dataset)
        training_losses.append(avg_epoch_loss)
        print(f"--- Epoch {epoch+1} finished. Average Loss: {avg_epoch_loss:.6f} ---")
        
    return training_losses

training_losses = train_model(model, dataloader, criterion, optimizer, EPOCHS)

print("\nTraining complete.")



Model initialized and sent to cpu.
Total parameters: 331,075
------------------------------
Epoch [1/5], Batch [0/36500], Loss: 0.130686
Epoch [1/5], Batch [10/36500], Loss: 0.106104
Epoch [1/5], Batch [20/36500], Loss: 0.035106
Epoch [1/5], Batch [30/36500], Loss: 0.037618
Epoch [1/5], Batch [40/36500], Loss: 0.023606
Epoch [1/5], Batch [50/36500], Loss: 0.042371
Epoch [1/5], Batch [60/36500], Loss: 0.020037
Epoch [1/5], Batch [70/36500], Loss: 0.016458
Epoch [1/5], Batch [80/36500], Loss: 0.032654
Epoch [1/5], Batch [90/36500], Loss: 0.029115
Epoch [1/5], Batch [100/36500], Loss: 0.012013
Epoch [1/5], Batch [110/36500], Loss: 0.020026
Epoch [1/5], Batch [120/36500], Loss: 0.033173
Epoch [1/5], Batch [130/36500], Loss: 0.024617
Epoch [1/5], Batch [140/36500], Loss: 0.011300
Epoch [1/5], Batch [150/36500], Loss: 0.011436
Epoch [1/5], Batch [160/36500], Loss: 0.016913
Epoch [1/5], Batch [170/36500], Loss: 0.013284
Epoch [1/5], Batch [180/36500], Loss: 0.015528
Epoch [1/5], Batch [190/3

In [9]:
# torch.save(model, f"model-v2")